In [5]:
#Loading the data
import os 
import csv

lines=[]
data_folder = 'C:\data\\'
file = '\driving_log.csv'
for child in os.listdir(data_folder):
    data_subfolder = os.path.join(data_folder, child)
    sub_file = data_subfolder + file
    with open(sub_file) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)
    print ("file: ", sub_file, ", data samples: ", len(lines))
            
print("total data samples: ", len(lines))


file:  C:\data\1\driving_log.csv , data samples:  8037
file:  C:\data\2\driving_log.csv , data samples:  9091
total data samples:  9091


In [11]:
import cv2
import numpy as np

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

def augmentation(images, measurments):
    aug_images, aug_measurments= [], []
    for image, measurment in zip(images, measurments):
        aug_images.append(cv2.flip(image,1))
        aug_measurments.append(measurment*-1.0)
    return aug_images, aug_measurments

def generator(samples, batch_size=32):
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, len(samples), batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurments = []
            for line in lines:

                image_center = cv2.imread(line[0])
                image_left = cv2.imread(line[1])
                image_right = cv2.imread(line[2])
                images.extend([image, image_left, image_right])

                steering = float(line[3])
                measurments.extend([steering, steering+0.1, steering-0.1])
                
                #augmented_images, augmented_measurments = augmentation([image, image_left, image_right], [steering, steering+0.1, steering-0.1])
                #images.extend(augmented_images)
                #measurments.extend(augmented_measurments)
                
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(measurments)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

#print("features Shape: ", train_generator)
#print("lables Shape: ", validation_generator)

In [59]:
#Augmentation - flip all images

aug_images, aug_measurments= [], []
for image, measurment in zip(images, measurments):
    aug_images.append(cv2.flip(image,1))
    aug_measurments.append(measurment*-1.0)
    

print("augmented features Shape: ", len(aug_images))
print("augmented lables Shape: ", len(aug_measurments))

augmented features Shape:  5406
augmented lables Shape:  5406


In [60]:
# Adding the augmented images to the train data

X_train = np.concatenate([X_train, aug_images])
y_train = np.concatenate([y_train, aug_measurments])
print(X_train.shape, y_train.shape)

(10812,) (10812,)


In [57]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from time import time
start_time = time()

dropout = 0.5

model = Sequential()
#normalization and mean zero - centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/255.0 -0.5, input_shape=(160,320,3))) 
#cropping2D layer
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160,320,3)))
#model.add(Flatten())
model.add(Dense(1))
model.add(Conv2D(24, kernel_size=(5, 5), padding='valid', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(dropout))
model.add(Conv2D(36, kernel_size=(5, 5), padding='valid', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(dropout))
model.add(Conv2D(48, kernel_size=(5, 5), padding='valid', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(dropout))
model.add(Flatten())
model.add(Dense(100,activation='elu'))
model.add(Dense(50,activation='elu'))
model.add(Dense(10,activation='elu'))
model.add(Dense(1,activation='elu'))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5, verbose = 1)
model.save('model.h5')

total_time = time() - start_time
minutes, seconds = divmod(total_time, 60)
print ("Total time for uploading data: ", minutes, "min, {:.0f}".format(seconds),  "s ")

C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:846: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected lambda_16_input to have 4 dimensions, but got array with shape (10812, 1)

In [41]:
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch = len(train_samples), 
    validation_data = validation_generator, nb_val_samples = len(validation_samples), nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

NameError: name 'train_generator' is not defined